Пример команды:
стой
вперед
влево
вправо
расжать клешню
сжать клешню
двигаться вперед
двигаться назад
двигаться вправо
двигаться влево

**Задача:**
Сгенерировать звуковые файлы для данного списка команд. Приблизительно по 50 команд для теста и попробовать распознавание в непрерывной речи.
Ключевое слово для распознавания, возмем к примеру, робот.

['робот   стой',
 'робот   вперед',
 'робот   влево',
 'робот   вправо',
 'робот   расжать клешню',
 'робот   сжать клешню',
 'робот   двигаться вперед',
 'робот   двигаться назад',
 'робот   двигаться вправо',
 'робот   двигаться влево'
]

In [ ]:
%%capture
!pip install omegaconf

In [ ]:
import os
import torch
from IPython.display import Audio
# Библиотека для воспроизведения звуковых файлов
import IPython.display as ipd
import zipfile
from tqdm.notebook import tqdm
import time
import librosa
import soundfile as sf

import subprocess
import tensorflow as tf
import tensorflow_hub as tf_hub
from omegaconf import OmegaConf

In [ ]:
  # Для каждого файла
#    for wavfile in tqdm(wavfiles, "Сохраняем векторы класса - '{}'".format(label)):

# Команды роботов

In [ ]:
list_command = {
                '0_robot':          'РОБОТ',
                '1_stop':           'СТОЙ',
                '2_forward':        'ВПЕРЕД',
                '3_backwards':      'НАЗАД',
                '4_left':           'ВЛЕВО',
                '5_right':          'ВПРАВО',
                '6_unclench':       'РАСЖАТЬ',
                '7_claw':           'КЛЕШНЮ',
                '8_squeeze':        'СЖАТЬ',
                '9_noise':          'ШУМ'
                }

classe = [
          'РОБОТ',
          'СТОЙ',
          'ВПЕРЕД',
          'НАЗАД',
          'ВЛЕВО',
          'ВПРАВО',
          'РАСЖАТЬ',
          'КЛЕШНЮ',
          'СЖАТЬ',
          'ШУМ'
]

In [ ]:
for i, class_name in enumerate(list_command):
  key = class_name  # ключ - это значение из перебора
  value = list_command[class_name]  # значение - это значение из словаря
  print(f'{i} {key} {value}')

In [ ]:
device = torch.device('cpu')
torch.set_num_threads(4)
local_file = 'model.pt'
speakers = ['aidar', 'baya', 'kseniya', 'xenia', 'eugene']

sample_rate = 24000  # 8000, 24000, 48000
n_files = 5   # Сколько файлов надо сгенерировать
parent_folder = '/content/data_train_robot'
put_accent = True
put_yo = True

language = 'ru'
model_id = 'v4_ru'

model, example_text = torch.hub.load(repo_or_dir = 'snakers4/silero-models',
                                     model = 'silero_tts',
                                     language = language,
                                     speaker = model_id)

In [ ]:
model.speakers

In [ ]:
# Просмотр доступных свойств и методов объекта model
p = [print(com) for com in dir(model)]

In [ ]:
help(model.apply_tts)

In [ ]:
def get_format_text(write_text, i):
    format_texts = [
               f'<speak><p>{write_text}</p></speak>',                                                 # Нормально
               f'<speak><p><prosody rate="x-slow">{write_text}</prosody></p></speak>',                # Медленно
               f'<speak><p><prosody rate="fast">{write_text}</prosody></p></speak>',                  # Довольно быстро
               f'<speak><p><prosody pitch="x-low">{write_text}</prosody></p></speak>',                # Тоном ниже
               f'<speak><p><prosody pitch="x-high">{write_text}</prosody></p></speak>',               # Тоном выше

               f'<speak><p><prosody rate="x-slow" pitch="x-low">{write_text}</prosody></p></speak>',  # Тоном ниже Медленно
               f'<speak><p><prosody rate="fast" pitch="x-high">{write_text}</prosody></p></speak>',   # Тоном выше Довольно быстро
               f'<speak><p><prosody rate="fast" pitch="x-low">{write_text}</prosody></p></speak>',    # Тоном ниже Довольно быстро
               f'<speak><p><prosody rate="x-slow" pitch="x-high">{write_text}</prosody></p></speak>', # Тоном выше Медленно

               f'<speak><p><break time="2000ms"/>{write_text}<break time="2000ms"/></p></speak>'      # Задерка перед и после 2 секунды
              ]
    return format_texts[i]

# Фиксация времени старта формирования выборки
curr_time = time.time()

# Проходим по списку команд
for i, class_name in tqdm(enumerate(list_command), total = len(list_command)):
    key = class_name  # ключ - это значение из перебора
    value = list_command[class_name]  # значение - это значение из словаря
    path_folder_class = os.path.join(parent_folder, key)

    # Проверяем, существует ли директория, и создаем её, если она не существует
    if not os.path.exists(path_folder_class):
        os.makedirs(path_folder_class)

    # Формируем голосовые команды с голосом случайного спикера
    k = 0
    for j in tqdm(range(len(speakers))):
        speaker = speakers[j]
        for d in range(10):
            ssml_text = get_format_text(value, d)
            audio = model.apply_tts(ssml_text = ssml_text,
                        speaker = speaker,
                        sample_rate = sample_rate,
                        put_accent = put_accent,
                        put_yo = put_yo)

            path_file_class = os.path.join(path_folder_class, f'{k}.wav')

            # Сохраните аудио в файл
            # Преобразование тензора в аудиофайл и сохранение
            sf.write(path_file_class, audio, sample_rate)  # sample_rate - частота дискретизации
            k += 1
    # Вывод информации о готовности обработки датасета
    print(f'Класс {key} готов -> {round(time.time() - curr_time)} c')
    curr_time = time.time()

In [ ]:
# Имя архива, в который будут сохранены файлы
zip_filename = 'data_train_robot.zip'

# Открываем архив для записи
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Рекурсивно перебираем все файлы и подпапки в указанной папке
    for root, dirs, files in os.walk(parent_folder):
        for file in files:
            # Полный путь к файлу
            file_path = os.path.join(root, file)
            # Относительный путь к файлу относительно parent_folder
            relative_path = os.path.relpath(file_path, parent_folder)
            # Добавляем файл в архив с сохранением структуры папок
            zipf.write(file_path, relative_path)

print(f'Архив "{zip_filename}" создан успешно.')

# Генерим случайные данные для теста

In [ ]:
list_command_genarated = [
    'робот   стой',
    'робот   вперед',
    'робот   влево',
    'робот   вправо',
    'робот   расжать клешню',
    'робот   сжать клешню',
    'робот   двигаться вперед',
    'робот   двигаться назад',
    'робот   двигаться вправо',
    'робот   двигаться влево' ]


Для генерации случайного текста воспользовался ChatGPT 3.5 и вот - романтическое:


Вот двадцать различных кусочков текста на разные темы на русском языке, каждый не более чем из 200 слов:

1. Мир вокруг нас так многообразен. От красоты закатов до бескрайних лесов, природа впечатляет своим разнообразием.

2. В мире искусства каждый может найти нечто особенное. Картины, скульптуры, музыка — они способны вдохновлять и трогать душу.

3. Завтрак с круассанами и кофе в Париже - мечта многих. Город влюбленных притягивает своей романтикой.

4. Научные открытия всегда волновали человечество. С каждым днем наука открывает перед нами новые горизонты.

5. Зимние сказки с детьми на лыжах, снежные баталии и катание на санках - зима наполняет радостью и весельем.

6. Вдохновение приходит внезапно. Стихи, мелодии, идеи могут родиться в самый неожиданный момент.

7. Путешествия расширяют наше видение мира. Новые культуры, вкусы и запахи всегда оставляют впечатление.

8. Семейные вечера с настольными играми - это отличный способ провести время вместе и укрепить связи.

9. Океан глубок и загадочен. Его воды скрывают множество тайн и красот.

10. Краски осени, когда деревья окрашиваются в золото и багрянец, создают неповторимую атмосферу.

11. Друзья всегда рядом, даже если они находятся далеко. Они дарят поддержку и понимание.

12. Вкусный ужин в хорошей компании - это искусство. Гурманы знают, что это значит.

13. Улыбка ребенка - самое драгоценное сокровище. Родительская любовь непередаваема словами.

14. Любовь к книгам - это путешествие в другие миры, встреча с интересными персонажами и умные размышления.

15. Смотреть на звезды ночью и мечтать о бескрайних космических просторах - это вечная тайна вселенной.

16. Моменты тишины и умиротворения важны для души. Они позволяют нам находить в себе силы.

17. Городской пейзаж с высоты птичьего полета всегда впечатляет своей красотой и динамикой.

18. Встречи с давними друзьями наполняют наши дни радостью и воспоминаниями.

19. Загадки и головоломки тренируют ум и помогают развивать аналитическое мышление.

20. Закаты на море создают неповторимую атмосферу романтики и спокойствия.

Надеюсь, что эти кусочки текста вдохновят вас на размышления и воображение.

Текст не слишком длинный - пока так.

In [ ]:
text_test = [
f'Мир вокруг нас так многообразен. От красоты закатов  {list_command_genarated[6]} до бескрайних лесов, природа впечатляет своим разнообразием',
f'В мире искусства каждый может найти нечто особенное. Картины,  {list_command_genarated[7]} скульптуры, музыка — они способны вдохновлять и трогать душу',
f'Завтрак с круассанами и кофе в  {list_command_genarated[4]} Париже - мечта многих. Город влюбленных притягивает своей романтикой',
f'Научные открытия всегда волновали человечество.  {list_command_genarated[2]} С каждым днем наука открывает перед нами новые горизонты',
f'Зимние сказки с детьми на лыжах,  {list_command_genarated[7]} снежные баталии и катание на санках - зима наполняет радостью и весельем',
f'Вдохновение приходит внезапно. Стихи,  {list_command_genarated[6]} мелодии, идеи могут родиться в самый неожиданный момент',
f'Путешествия расширяют наше видение мира. Новые культуры,  {list_command_genarated[8]} вкусы и запахи всегда оставляют впечатление',
f'Семейные вечера с  {list_command_genarated[5]} настольными играми - это отличный способ провести время вместе и укрепить связи',
f'Океан глубок и загадочен.  {list_command_genarated[3]} Его воды скрывают множество тайн и красот',
f'Краски осени, когда деревья окрашиваются в {list_command_genarated[9]}  золото и багрянец, создают неповторимую атмосферу',
f'Друзья всегда рядом, даже если они находятся далеко. Они дарят поддержку и понимание',
f'Вкусный ужин в хорошей  {list_command_genarated[1]} компании - это искусство. Гурманы знают, что это значит',
f'Улыбка ребенка - самое драгоценное сокровище.  {list_command_genarated[6]} Родительская любовь непередаваема словами',
f'Любовь к книгам - это  {list_command_genarated[2]} путешествие в другие миры, встреча с интересными персонажами и умные размышления',
f'Смотреть на звезды ночью и мечтать о бескрайних космических {list_command_genarated[0]}  просторах - это вечная тайна вселенной',
f'Моменты тишины и умиротворения {list_command_genarated[3]}  важны для души. Они позволяют нам находить в себе силы',
f'Городской пейзаж с высоты птичьего полета всегда впечатляет своей красотой {list_command_genarated[4]}  и динамикой',
f'Встречи с давними  {list_command_genarated[1]} друзьями наполняют наши дни радостью и воспоминаниями',
f'Загадки и головоломки тренируют ум и помогают развивать {list_command_genarated[8]}  аналитическое мышление',
f'Закаты на море создают  {list_command_genarated[5]} неповторимую атмосферу романтики и спокойствия']

In [ ]:
p = [print(text) for text in text_test]

In [ ]:
device = torch.device('cpu')
torch.set_num_threads(4)
local_file = 'model.pt'
speaker = 'random' # 'aidar', 'baya', 'kseniya', 'xenia', 'random'
sample_rate = 24000  # 8000, 24000, 48000
n_files = 50   # Сколько файлов надо сгенерировать
parent_folder = '/content/data_test_robot'
put_accent = True
put_yo = True

language = 'ru'
model_id = 'v4_ru'

model, example_text = torch.hub.load(repo_or_dir = 'snakers4/silero-models',
                                     model = 'silero_tts',
                                     language = language,
                                     speaker = model_id)

# Проходим по списку команд
for i in tqdm(range(len(text_test))):
    example_text = text_test[i]

    # Проверяем, существует ли директория, и создаем её, если она не существует
    if not os.path.exists(parent_folder):
        os.makedirs(parent_folder)

    audio = model.apply_tts(text = example_text,
                        speaker = speaker,
                        sample_rate = sample_rate,
                        put_accent = put_accent,
                        put_yo = put_yo)

    path_file_class = os.path.join(parent_folder, f'{i}.wav')

    # Сохраните аудио в файл
    # Преобразование тензора в аудиофайл и сохранение
    sf.write(path_file_class, audio, sample_rate)  # sample_rate - частота дискретизации

In [ ]:
# Имя архива, в который будут сохранены файлы
zip_filename = 'data_test_robot.zip'

# Открываем архив для записи
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Рекурсивно перебираем все файлы и подпапки в указанной папке
    for root, dirs, files in os.walk(parent_folder):
        for file in files:
            # Полный путь к файлу
            file_path = os.path.join(root, file)
            # Относительный путь к файлу относительно parent_folder
            relative_path = os.path.relpath(file_path, parent_folder)
            # Добавляем файл в архив с сохранением структуры папок
            zipf.write(file_path, relative_path)

print(f'Архив "{zip_filename}" создан успешно.')